Warning this example will process data in the Google Cloud but should be well within the free tier
Data is approximately 4.97 GB in size at time of creation of this notebook
https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales

Goal: We would like to create a dataset to predict a stores sales in 2019. This dataset will have two features
1) The total sales in 2018
2) The percent change in sales in 2018 vs 2017

In [1]:
from feature_graph.base import FeatureDAG
from feature_graph.bigquery_node import BigQueryNode

project = "mintos-200210"

with FeatureDAG() as dag:
    
    features_for_model_node = BigQueryNode(name="features_for_model", 
                     query="""
   CREATE OR REPLACE TABLE `mintos-200210.feature_graph_test.features_for_model`
       AS
    SELECT sy.store_name
         , sy.sales_year_2018_change
         , sy.sales_year_2018_dollars
         , sy.sales_year_2019_dollars
         , COALESCE(nc.new_brands_sold_in_2018, 0) as new_brands_sold_in_2018
      FROM `mintos-200210.feature_graph_test.sales_year_features` sy
 LEFT JOIN `mintos-200210.feature_graph_test.new_categories_2018` nc
           ON sy.store_name = nc.store_name                     
                     """, 
                     project=project) 
    
    sales_base_node = BigQueryNode(name="sales_base", 
                              query="""
        CREATE OR REPLACE TABLE `mintos-200210.feature_graph_test.sales_base`
           AS
        SELECT store_name
             , category_name
             , EXTRACT(YEAR FROM date) as sales_year
             , ROUND(SUM(sale_dollars), 2) as sales_year_dollars
         FROM `bigquery-public-data.iowa_liquor_sales.sales`
        WHERE EXTRACT(YEAR FROM date) IN (2017, 2018, 2019)
        GROUP BY 1,2,3            
                              """, 
                              project=project)
    
    sales_features_base_node = BigQueryNode(name="sales_features_base",
                     query="""
        CREATE OR REPLACE TABLE `mintos-200210.feature_graph_test.sales_features_base`
           AS
        SELECT store_name
             , EXTRACT(YEAR FROM date) as sales_year
             , ROUND(SUM(sale_dollars), 2) as sales_year_dollars
          FROM `bigquery-public-data.iowa_liquor_sales.sales`
         GROUP BY 1,2                     
                     """, 
                     project=project)
    
    sales_year_features_node = BigQueryNode(name="sales_year_features", 
                     query="""
                     
       CREATE OR REPLACE TABLE `mintos-200210.feature_graph_test.sales_year_features`
           AS
        SELECT store_name
             , sales_year_2019_dollars
             , sales_year_2018_dollars
             , CASE WHEN sales_year_2017_dollars > 0
                    THEN (sales_year_2018_dollars - sales_year_2017_dollars) 
                           / sales_year_2017_dollars
                    ELSE 1
                END  as sales_year_2018_change
          FROM (
                SELECT store_name
                     , COALESCE(CASE WHEN sales_year = 2017 THEN sales_year_dollars END, 0) as sales_year_2017_dollars
                     , COALESCE(CASE WHEN sales_year = 2018 THEN sales_year_dollars END, 0) as sales_year_2018_dollars
                     , COALESCE(CASE WHEN sales_year = 2019 THEN sales_year_dollars END, 0) as sales_year_2019_dollars
                 FROM `mintos-200210.feature_graph_test.sales_features_base`
               )                
                     """, 
                     project=project)
    
    new_categories_2018_node = BigQueryNode(name="new_categories_2018", 
                     query="""
                     
        CREATE OR REPLACE TABLE `mintos-200210.feature_graph_test.new_categories_2018`
           AS
        SELECT sb_2018.store_name
             , COUNT(CASE WHEN sb_2018.category_name IS NOT NULL AND sb_2017.category_name IS NULL THEN 1 END) new_brands_sold_in_2018
         FROM (
               SELECT store_name
                    , category_name
                 FROM `mintos-200210.feature_graph_test.sales_base`
                 WHERE sales_year = 2018
               ) sb_2018
    LEFT JOIN  (
               SELECT store_name
                    , category_name
                 FROM `mintos-200210.feature_graph_test.sales_base`
                 WHERE sales_year = 2017
               ) sb_2017
               ON sb_2018.store_name = sb_2017.store_name
                  AND sb_2018.category_name = sb_2017.category_name
         GROUP BY 1
                     """, 
                     project=project)

    sales_base_node >> sales_features_base_node
    sales_features_base_node >> sales_year_features_node
    sales_base_node >> new_categories_2018_node
    sales_year_features_node >> features_for_model_node
    new_categories_2018_node >> features_for_model_node

display(dag)
dag.run_feature_graph()

ModuleNotFoundError: No module named 'feature_graph'